In [28]:
import pennylane as qml
from pennylane import numpy as np

In [27]:
a = [[0.66554315], [0.66554315, 0.81540048], [0.73839569, 0.82715646, 0.76431838, 0.90401019]]

In [30]:
dev = qml.device("default.qubit", wires=3)

In [ ]:
@qml.qnode(dev)
def circuit():

    qml.RY(np.pi, wires=[0])

    qml.CRY(np.pi / 2, wires=[0, 1])

    qml.X(wires=[1])
    qml.CRY(np.pi / 2, wires=[0, 1])
    qml.X(wires=[1])

    qml.ctrl(qml.RX, control=[0, 1])(np.pi / 4, wires=2)

    qml.X(wires=[1])
    qml.ctrl(qml.RX, control=[0, 1])(np.pi / 4, wires=2)
    qml.X(wires=[1])

    qml.X(wires=[0])
    qml.ctrl(qml.RX, control=[0, 1])(np.pi / 4, wires=2)
    qml.X(wires=[0])

    qml.X(wires=[0])
    qml.X(wires=[1])
    qml.ctrl(qml.RX, control=[0, 1])(np.pi / 4, wires=2)
    qml.X(wires=[0])
    qml.X(wires=[1])

    return qml.state()
    

In [36]:
drawer = qml.draw(circuit)
print(drawer())

0: ──RY(3.14)─╭●───────────╭●───────────╭●───────────╭●─────────X─╭●─────────X──X─╭●─────────X─┤
1: ───────────╰RY(1.57)──X─╰RY(1.57)──X─├●─────────X─├●─────────X─├●─────────X────├●─────────X─┤
2: ─────────────────────────────────────╰RX(0.79)────╰RX(0.79)────╰RX(0.79)───────╰RX(0.79)────┤

   State
   State
   State


In [39]:
# Execute the circuit
state = circuit()

# Print the resulting quantum state
print("\nFinal Statevector:")
for idx, amp in enumerate(state):
    print(f"State |{idx:03b}>: Amplitude = {amp}")


Final Statevector:
State |000>: Amplitude = (5.657130561438501e-17+0j)
State |001>: Amplitude = -2.3432602026631493e-17j
State |010>: Amplitude = 0j
State |011>: Amplitude = 0j
State |100>: Amplitude = (0.9238795325112867+0j)
State |101>: Amplitude = -0.3826834323650898j
State |110>: Amplitude = 0j
State |111>: Amplitude = 0j


In [41]:
import pennylane as qml
from pennylane import numpy as np

# Grayscale values for a 2x2 image
grayscale_values = np.array([64, 128, 192, 255])
M = 256

# Normalize the grayscale values to amplitudes
norm_factor = np.sqrt(np.sum(grayscale_values**2))
alphas = grayscale_values / norm_factor

# Define device
dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def naqss():
    # Encode pixel (0, 0)
    qml.RY(2 * np.arcsin(alphas[0]), wires=2)

    # Encode pixel (0, 1)
    qml.ctrl(qml.RY, control=[0])(2 * np.arcsin(alphas[1]), wires=2)

    # Encode pixel (1, 0)
    qml.ctrl(qml.RY, control=[1])(2 * np.arcsin(alphas[2]), wires=2)

    # Encode pixel (1, 1)
    qml.ctrl(qml.RY, control=[0, 1])(2 * np.arcsin(alphas[3]), wires=2)

    return qml.state()

# Execute and visualize the state
state = naqss()
print("\nFinal Statevector:")
for idx, amp in enumerate(state):
    print(f"State |{idx:03b}>: Amplitude = {amp}")



Final Statevector:
State |000>: Amplitude = (0.983121289665021+0j)
State |001>: Amplitude = (0.18295499393945458+0j)
State |010>: Amplitude = 0j
State |011>: Amplitude = 0j
State |100>: Amplitude = 0j
State |101>: Amplitude = 0j
State |110>: Amplitude = 0j
State |111>: Amplitude = 0j
